Is direct exchange responsible for 90° AFM interactions between octahedrally connected TM sites?
-> analyze distances between magnetic sites as a function of both ionic radii

- assumption: correct oxidation state from BVA / pymatgen guessing method
- not included in radii: high-spin / low-spin and local environment as either no reliable info from structure (hs vs. ls) or data not available for all coordination environments
- missing ion radius data for connected TM in octahedra: only two compounds #0.447 (Mn1+) and #1.722 with "Ru4.5+" -> skipped in analysis

- further important: MAGNDATA atomic positions often taken from diffraction experiment above magnetic ordering temperature -> may influence trends (structural transition accompanying magnetic transition may not be considered)

In [1]:
import json
import math
from monty.json import MontyDecoder
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from pymatgen.core import Element, Species

from utils_kga.statistical_analysis.get_spin_and_bond_angle_statistics import get_bond_angle_interval_statistics
from utils_kga.general import pretty_plot

In [2]:
def weighted_lower_median(pairs):
    thr = sum([p[1] for p in pairs]) / 2
    pairs.sort(key = lambda p: p[0])
    w_sums = 0
    for a, w in pairs:
        w_sums += w
        if len(pairs) % 2 != 0 and w_sums > thr:
            return a
        elif len(pairs) % 2 == 0 and w_sums >= thr:
            return a

In [3]:
def get_radii_resolved_distance_metric(ang_df: pd.DataFrame):
    ang_df["distance_difference_to_radii_sum_ratio"] = ang_df.apply(
        lambda x: ((x["distance"] - (x["site_ionic_radius"] + x["site_to_ionic_radius"])) / (x["site_ionic_radius"] + x["site_to_ionic_radius"])), axis=1)

In [4]:
# Load edge-dfs
with open("data/dfs_of_magnetic_edge_information.json") as f:
    dict_all_stats = json.load(f)
all_stats = {key: pd.DataFrame.from_dict(df) for key, df in dict_all_stats.items()}

# For metadata filtering
with open("../../data_retrieval_and_preprocessing_MAGNDATA/data/df_grouped_and_chosen_commensurate_MAGNDATA.json") as f:
    df = json.load(f, cls=MontyDecoder)
    
description = [
    "all edges with TM octahedra at both nodes",
    "all oxygen edges with TM octahedra at both nodes",
]
colors = px.colors.qualitative.Plotly

plot_dir = "plots/TM_octahedra_analysis_distances"
os.makedirs(plot_dir, exist_ok=True)

In [5]:
skip_ids = []
for md_id, ang_df in all_stats.items():
    ang_df["spin_angle"] = ang_df["spin_angle"].apply(lambda x: round(x, 0))
    ang_df["site_is_tm"] = ang_df["site_element"].apply(lambda el: Element(el).is_transition_metal)
    ang_df["site_to_is_tm"] = ang_df["site_to_element"].apply(lambda el: Element(el).is_transition_metal)
    ang_df["ligand_el_set"] = ang_df["ligand_elements"].apply(lambda ls: set(ls))
    ang_df["av_bond_angle"] = ang_df["mag_ligand_mag_angle"].apply(lambda ls: np.mean(np.array(ls)))
    
    ang_df["site_species"] = ang_df.apply(lambda x: x["site_element"] + str(x["site_oxidation"]) + "+", axis=1)
    ang_df["site_ionic_radius"] = ang_df.apply(lambda x: Species(symbol=x["site_element"], oxidation_state=x["site_oxidation"]).ionic_radius, axis=1)
    if None in ang_df["site_ionic_radius"].values or True in [math.isnan(v) for v in ang_df["site_ionic_radius"].values]: 
        if not ang_df.loc[(~ang_df["site_ionic_radius"].notna()) 
                                    & (ang_df["site_ce"]=="O:6") 
                                    & (ang_df["site_to_ce"]=="O:6")
                                    & (ang_df["site_is_tm"])
                                    & (ang_df["site_to_is_tm"])].empty:# check for missing ionic radii of connected TM sites in octahedra 
            print(md_id, set(ang_df.loc[(~ang_df["site_ionic_radius"].notna()) 
                                        & (ang_df["site_ce"]=="O:6") 
                                        & (ang_df["site_to_ce"]=="O:6")
                                        & (ang_df["site_is_tm"])
                                        & (ang_df["site_to_is_tm"])]["site_species"].values))
            skip_ids.append(md_id)
    
    ang_df["site_to_species"] = ang_df.apply(lambda x: x["site_to_element"] + str(x["site_to_oxidation"]) + "+", axis=1)
    ang_df["site_to_ionic_radius"] = ang_df.apply(lambda x: Species(symbol=x["site_to_element"], oxidation_state=x["site_to_oxidation"]).ionic_radius, axis=1)

0.447 {'Mn1+'}
1.722 {'Ru4.5+'}


In [8]:
stats_dict_all = {}
bond_angle_ranges = [(89, 91), (85, 95), (80, 100), (75, 105)]
y_lim_range = dict(zip(bond_angle_ranges, [40, 120, 120, 120]))  # for plotting of average and median
normalize_string = "absolute occurrences"
for bond_angle_range90 in bond_angle_ranges:
    for ligand_multiplicity_bool, ligand_multiplicity_string in zip([False], ["no ligand multiplicity included"]):
        for data_string in description:
            metrics = {"distance_difference_to_radii_sum_ratio": {"FM": [], "AFM": []}}
            compounds = {"FM": [], "AFM": []}
            for md_id, ang_df in all_stats.items():
                if md_id not in skip_ids:
                    subdf = ang_df.loc[(ang_df["site_ce"]=="O:6") 
                           & (ang_df["site_to_ce"]=="O:6")
                           & (ang_df["site_is_tm"]) 
                           & (ang_df["site_to_is_tm"])
                    ]
                    if "oxygen" in data_string:
                        subdf = subdf.loc[subdf["ligand_el_set"]=={"O"}]
                        
                    n_lattice_points = df.at[md_id, "n_lattice_points"]
                    
                    for magnet_type, spin_angle_condition in zip(["FM", "AFM"], [subdf["spin_angle"]<=10.0, subdf["spin_angle"]>=170.0]):
                        magnet_df = subdf.loc[spin_angle_condition]
                        
                        try:
                            get_radii_resolved_distance_metric(magnet_df)
                        except TypeError:  # no radii present -> already checked above that only 2 structures of KGA-relevant interactions concerned
                            continue
                            
                        for metric in metrics:
                            occus = get_bond_angle_interval_statistics(df=magnet_df, include_ligand_multiplicity=ligand_multiplicity_bool,
                                                                       analyze_column=metric, n_lattice_points=n_lattice_points,
                                                                       bond_angle_interval=bond_angle_range90)
                            if occus:
                                metrics[metric][magnet_type].extend(occus)
                                compounds[magnet_type].append(md_id)
            n_compounds = {k: len(v) for k, v in compounds.items()}
            for metric, metric_dict in metrics.items():
                stats_dict = {}
                one_d_fig = go.Figure(layout=go.Layout(xaxis=go.layout.XAxis(title=metric),
                                                       yaxis=go.layout.YAxis(title="Occurrence"),
                                                       title=f"{metric}, {data_string}, {ligand_multiplicity_string}, {normalize_string}, bond angles={bond_angle_range90}, compounds={n_compounds}"))
                for mag_idx, mag_string in enumerate(["FM", "AFM"]):
                    mag_df = pd.DataFrame(columns=["distance_metric", "occurrence"], data=metric_dict[mag_string])
                    av = np.average(mag_df["distance_metric"], weights=mag_df["occurrence"])
                    median = weighted_lower_median(metric_dict[mag_string])
                    one_d_fig.add_trace(go.Histogram(
                        histfunc="sum",
                        x=mag_df["distance_metric"].values,
                        y=mag_df["occurrence"].values,
                        name=mag_string,
                        marker_color= colors[mag_idx]
                    ))
                    for stat, stat_string in zip([av, median], ["av", "median"]):
                        one_d_fig.add_trace(go.Scatter(
                            x=[stat, stat],
                            y=[0, y_lim_range[bond_angle_range90]],
                            mode="lines",
                            marker_color=colors[mag_idx+2],
                            name=mag_string + f" {stat_string} ({round(stat, 2)})"
                        ))
                        stats_dict[f"{mag_string} {stat_string}"] = stat
                    stats_dict_all.update({f"{bond_angle_range90}-{data_string}-{metric}": stats_dict})    
                one_d_fig = pretty_plot(one_d_fig)
                one_d_fig.update_layout(title=dict(font=dict(size=10)))
                one_d_fig.write_html(os.path.join(plot_dir, f"{metric}_{data_string}_{bond_angle_range90[0]}-{bond_angle_range90[1]}-deg_bond_angles.html"))

                # Save example to pdf
                if bond_angle_range90 == (85, 95) and data_string == "all edges with TM octahedra at both nodes":
                    one_d_fig = go.Figure(layout=go.Layout(xaxis=go.layout.XAxis(title=metric),
                                                       yaxis=go.layout.YAxis(title="Occurrence"),
                                                       title=f"{metric}, {data_string}, {ligand_multiplicity_string}, {normalize_string}, bond angles={bond_angle_range90}, compounds={n_compounds}"))
                    for mag_string in ["FM", "AFM"]:
                        one_d_fig = go.Figure(layout=go.Layout(xaxis=go.layout.XAxis(title=metric),
                                                       yaxis=go.layout.YAxis(title="Occurrence"),
                                                       title=f"{metric}, {data_string}, {ligand_multiplicity_string}, {normalize_string}, bond angles={bond_angle_range90}, {mag_string}"))
                        mag_df = pd.DataFrame(columns=["distance_metric", "occurrence"], data=metric_dict[mag_string])
                        av = np.average(mag_df["distance_metric"], weights=mag_df["occurrence"])
                        median = weighted_lower_median(metric_dict[mag_string])
                        one_d_fig.add_trace(go.Histogram(
                            histfunc="sum",
                            x=mag_df["distance_metric"].values,
                            y=mag_df["occurrence"].values,
                            name=mag_string,
                            showlegend=False,
                            marker_color="#025268"
                        ))
                        for stat, stat_string in zip([av, median], ["av", "median"]):
                            one_d_fig.add_trace(go.Scatter(
                                x=[stat, stat],
                                y=[0, 140],
                                mode="lines",
                                name=mag_string + f" {stat_string} ({round(stat, 2)})",
                                showlegend=False,
                                line=dict(width=2)
                            ))
                        one_d_fig = pretty_plot(one_d_fig)
                        one_d_fig.update_layout(title=dict(font=dict(size=10)), legend=dict(font=dict(size=10)))
                        one_d_fig.update_layout(xaxis_range=[0.4, 1.7])
                        one_d_fig.update_layout(yaxis_range=[0, 140])
                        one_d_fig.update_yaxes(zeroline=False)
                        one_d_fig.write_image(os.path.join(plot_dir, f"{metric}_{data_string}_{bond_angle_range90[0]}-{bond_angle_range90[1]}-deg_bond_angles_{mag_string}.pdf"))

In [9]:
av_median_df = pd.DataFrame(stats_dict_all).T
av_median_df

,FM av,FM median,AFM av,AFM median
"(89, 91)-all edges with TM octahedra at both nodes-distance_difference_to_radii_sum_ratio",0.885019,0.84,0.818374,0.77
"(89, 91)-all oxygen edges with TM octahedra at both nodes-distance_difference_to_radii_sum_ratio",0.740868,0.75,0.726827,0.70
"(85, 95)-all edges with TM octahedra at both nodes-distance_difference_to_radii_sum_ratio",0.926402,0.90,0.800077,0.78
"(85, 95)-all oxygen edges with TM octahedra at both nodes-distance_difference_to_radii_sum_ratio",0.829479,0.84,0.756187,0.75
"(80, 100)-all edges with TM octahedra at both nodes-distance_difference_to_radii_sum_ratio",0.921468,0.90,0.854046,0.89
"(80, 100)-all oxygen edges with TM octahedra at both nodes-distance_difference_to_radii_sum_ratio",0.850085,0.86,0.814915,0.81
"(75, 105)-all edges with TM octahedra at both nodes-distance_difference_to_radii_sum_ratio",0.922749,0.90,0.884133,0.90
"(75, 105)-all oxygen edges with TM octahedra at both nodes-distance_difference_to_radii_sum_ratio",0.855485,0.85,0.848758,0.88
